<a href="https://colab.research.google.com/github/Strojove-uceni/23206-final-pose-estimation-for-swing-improvement/blob/main/accuracies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2 #opencv
import mediapipe as mp
import math
import csv
import datetime
import pandas as pd
import os
from mediapipe_estimate import estimate
pd.options.mode.chained_assignment = None

In [ ]:
directory_path = 'results_wrist_elbow/'

In [ ]:
test_dat=data=pd.read_csv('golfDB_front_orig.csv',index_col='id')
test_dat=test_dat[['youtube_id','events_2','events_5','events_7']]
test_dat['events_5']=test_dat['events_5']-test_dat['events_2']+15
test_dat['events_7']=test_dat['events_7']-test_dat['events_2']+15

test_dat=test_dat.drop(['events_2'],axis=1)
test_dat

,youtube_id,events_5,events_7
id,,,
8,-M5SITXMA2Y,45,53
10,gOBVh7fzyZo,39,47
14,vN3Uc_EhnnY,41,49
19,iW323nsTGtU,49,57
21,xD6KDqPF9cc,39,45
...,...,...,...
1370,g90r9cs1tTw,37,43
1372,eNlBbMI-MhQ,49,59
1374,6K1FD0sOwoI,54,63


In [ ]:
name_list=list('output_video_'+pd.unique(test_dat.youtube_id))
all=len(name_list)
correct_contact=0
correct_top=0
correct_address=0
error_sec=[]
error_frames=[]
for fl in name_list:

    try:
        video_files = [file for file in os.listdir(directory_path) if fl in file and file.endswith('.mp4')]

        # Full paths
        video_path =directory_path +video_files[0]

        data=pd.read_csv('results_wrist_elbow/variables_'+fl+'.csv')
        cap = cv2.VideoCapture(video_path)

    # Check if video opened successfully
        if not cap.isOpened():
            print(f"Error opening video file {fl}")
            continue

            # Set the frame position


        #consider only time between backswing and finish
        halfway_back_ind=data['right_wrist_x'].idxmin()
        halfway_front_ind=data.right_wrist_x[data.index>halfway_back_ind].idxmax()
        middle_data=data[(data.index>halfway_back_ind)&(data.index<halfway_front_ind)]

        #find moment of ball contact as the lowest wrist point on y
        contact_frame=middle_data[middle_data['right_wrist_y']==middle_data['right_wrist_y'].max()].index

        #isolate only backswing data
        back_data=data[(data.index<contact_frame.item())]

        #find moment of top of backswing as the highest wrist point on y
        top_backswing_frame=back_data[back_data['right_wrist_y']==back_data['right_wrist_y'].min()].index

        #find moment before start of the swing as the lowest wrist point on y before going halfway back
        halfway_back_data=data[data['right_wrist_x'].index<halfway_back_ind]
        address_frame=halfway_back_data[halfway_back_data['right_wrist_y']==halfway_back_data['right_wrist_y'].max()].index

        #extract the gt from golfDB
        yt_id='_'.join(fl.split('_')[2::])
        gt_contact=test_dat[test_dat.youtube_id==yt_id].events_7.item()
        gt_top=test_dat[test_dat.youtube_id==yt_id].events_5.item()
        gt_address=15

        #calculte tolerance as per golfDB paper (fps=30)
        tolerance_hit=2*(gt_contact//30)
        tolerance_top=2*(gt_top//30)
        tolerance_address=2*(gt_address//30)
        swing_part=address_frame.item()-3


        cap.set(cv2.CAP_PROP_POS_FRAMES, swing_part)

        # Read the frame
        ret, frame = cap.read()

        # Check if frame is read correctly
        if not ret:
            print(f"Error reading frame from {video_path}")
            continue

        # Save the frame as an image
        frame_path =os.path.splitext(video_path)[0].split("/")[0]+'/frames_address/'+os.path.splitext(video_path)[0].split("/")[1]+f'_frame_{swing_part}.jpg'
        cv2.imwrite(frame_path, frame)

        # Release the video capture object
        cap.release()

        if gt_contact-tolerance_hit<=contact_frame.item()<=tolerance_hit+gt_contact:
            correct_contact+=1
            print('correct hit esitmate',correct_contact,' estimated [s]:',data[data.index==contact_frame.item()].timestamp.item(),'gt [s]:', data[data.index==gt_contact].timestamp.item(),', tolerance [frames]:',tolerance_hit, ', frame difference:', abs(contact_frame.item()-gt_contact))
        else:
            print('incorrect hit estimate',correct_contact,' estimated [s]:',data[data.index==contact_frame.item()].timestamp.item(),', gt [s]:', data[data.index==gt_contact].timestamp.item(),', tolerance [frames]:',tolerance_hit,', frame difference:', abs(contact_frame.item()-gt_contact))
            error_sec.append(abs(data[data.index==contact_frame.item()].timestamp.item()-data[data.index==gt_contact].timestamp.item()))
        if gt_top-tolerance_top<=top_backswing_frame.item()<=tolerance_top+gt_top:
            correct_top+=1
            print('correct top esitmate',correct_top,' estimated [s]:',data[data.index==top_backswing_frame.item()].timestamp.item(),'gt [s]:', data[data.index==gt_top].timestamp.item(),', tolerance [frames]:',tolerance_top, ', frame difference:', abs(top_backswing_frame.item()-gt_top))
        else:
            print('icorrect top esitmate',correct_top,' estimated [s]:',data[data.index==top_backswing_frame.item()].timestamp.item(),'gt [s]:', data[data.index==gt_top].timestamp.item(),', tolerance [frames]:',tolerance_top, ', frame difference:', abs(top_backswing_frame.item()-gt_top))

        if gt_address-tolerance_address<=address_frame.item()<=tolerance_address+gt_address:
            correct_address+=1
           print('correct address esitmate',correct_address,' estimated [s]:',data[data.index==address_frame.item()].timestamp.item(),'gt [s]:', data[data.index==gt_address].timestamp.item(),', tolerance [frames]:',tolerance_address, ', frame difference:', abs(address_frame.item()-gt_address))
        else:
           print('icorrect address esitmate',correct_address,' estimated [s]:',data[data.index==address_frame.item()].timestamp.item(),'gt [s]:', data[data.index==gt_address].timestamp.item(),', tolerance [frames]:',tolerance_address, ', frame difference:', abs(address_frame.item()-gt_address))
        print('----------------------------------------------------------------------------------------------------')

    except Exception as e:
        print(e,fl)
        print('----------------------------------------------------------------------------------------------------')
        all=all-1



accuracy_contact=correct_contact/all
accuracy_top=correct_top/all
accuracy_address=correct_address/all

print(f'contatct estimation accuracy: {accuracy_contact}, top estimation accuracy: {accuracy_top}, address estimation accuracy: {accuracy_address}')
#print(f'contatct estimation accuracy: {accuracy_contact}')

attempt to get argmin of an empty sequence output_video_vN3Uc_EhnnY
----------------------------------------------------------------------------------------------------
can only convert an array of size 1 to a Python scalar output_video_j6h-3Ez0714
----------------------------------------------------------------------------------------------------
attempt to get argmin of an empty sequence output_video_8n9BBNEj3yk
----------------------------------------------------------------------------------------------------
attempt to get argmin of an empty sequence output_video_FPvudrvNw6A
----------------------------------------------------------------------------------------------------
attempt to get argmin of an empty sequence output_video_bF8A2zdAfAs
----------------------------------------------------------------------------------------------------
can only convert an array of size 1 to a Python scalar output_video_04d08bM6-6U
---------------------------------------------------------------

In [ ]:
# data=pd.read_csv('results/variables_'+fl+'.csv')
# halfway_back_ind=data['right_wrist_x'].idxmin()
# data_back=data[data.index>halfway_back_ind]
# data['sum_wrists_hit']=data['right_wrist_y']+data.timestamp
# ev_6=data[data['sum_wrists_hit']==data['sum_wrists_hit'].max()].index
error_sec
for sec in error_sec:
    print(sec*30)


In [ ]:
os.path.splitext(video_path)[0].split("/")[0]+'/frames/'+os.path.splitext(video_path)[0].split("/")[1]+f'_frame_{contact_frame.item()}.jpg'

'results_wrist_elbow/frames/output_video_TtxGGIzrqWI_out_2023-11-26_10-47-42_frame_54.jpg'